In [ ]:
# import requests
# import json

# # Define the request components
# url = "http://localhost:8000/apps/healthcare-patient-id/users/u_123/sessions/s_123"
# headers = {"Content-Type": "application/json"}
# payload = {"state": {"email": "julia@email.com"}}

# try:
#     # Make the POST request
#     response = requests.post(url, headers=headers, json=payload)

#     # Raise an exception for bad status codes (4xx or 5xx)
#     response.raise_for_status()

#     # Print the successful response from the server
#     print(f"✅ Success! Status Code: {response.status_code}")
#     print("Response JSON:")
#     # Use json.dumps for pretty printing
#     print(json.dumps(response.json(), indent=4))

# except requests.exceptions.HTTPError as http_err:
#     print(f"❌ HTTP error occurred: {http_err}")
#     print(f"Response content: {response.text}")
# except requests.exceptions.RequestException as err:
#     print(f"❌ An error occurred: {err}")

❌ HTTP error occurred: 400 Client Error: Bad Request for url: http://localhost:8000/apps/healthcare-patient-id/users/u_123/sessions/s_123
Response content: {"detail":"Session already exists: s_123"}


In [ ]:
# from google.adk.sessions import InMemorySessionService, Session
# from google.adk.events import Event, EventActions
# from google.genai.types import Part, Content
# import time

# # --- Setup ---
# session_service = InMemorySessionService()
# app_name, user_id, session_id = "healthcare-patient-id", "u_123", "s_123"
# session = await session_service.create_session(
#     app_name=app_name,
#     user_id=user_id,
#     session_id=session_id,
#     state={"email": "julia@email.com"}

# )
# print(f"Initial state: {session.state}")

Initial state: {'email': 'julia@email.com'}


In [18]:
from google.adk.sessions import InMemorySessionService, Session

# Create a simple session to examine its properties
temp_service = InMemorySessionService()
example_session = await temp_service.create_session(
    app_name="my_app",
    user_id="example_user",
    state={"initial_key": "initial_value"} # State can be initialized
)

print(f"--- Examining Session Properties ---")
print(f"ID (`id`):                {example_session.id}")
print(f"Application Name (`app_name`): {example_session.app_name}")
print(f"User ID (`user_id`):         {example_session.user_id}")
print(f"State (`state`):           {example_session.state}") # Note: Only shows initial state here
print(f"Events (`events`):         {example_session.events}") # Initially empty
print(f"Last Update (`last_update_time`): {example_session.last_update_time:.2f}")
print(f"---------------------------------")

# Clean up (optional for this example)
temp_service = await temp_service.delete_session(app_name=example_session.app_name,
                            user_id=example_session.user_id, session_id=example_session.id)
print("The final status of temp_service - ", temp_service)

--- Examining Session Properties ---
ID (`id`):                3f306b07-0b53-4a9c-ad31-7afe2512b24c
Application Name (`app_name`): my_app
User ID (`user_id`):         example_user
State (`state`):           {'initial_key': 'initial_value'}
Events (`events`):         []
Last Update (`last_update_time`): 1755134482.46
---------------------------------
The final status of temp_service -  None


# Tools

In [5]:
import os
import datetime
import requests
import google.auth
import google.auth.transport.requests
from urllib.parse import quote
from typing import Optional
from dotenv import load_dotenv
import re

load_dotenv()
model_name = os.getenv("MODEL")
model_lite = os.getenv("MODEL_LITE")
PROJECT_ID = os.getenv('GOOGLE_CLOUD_PROJECT')
FHIR_LOCATION = os.getenv('FHIR_LOCATION')
FHIR_DATASET_ID = os.getenv('FHIR_DATASET_ID')
FHIR_STORE_ID = os.getenv('FHIR_STORE_ID')
BASE_FHIR_URL = f"https://healthcare.googleapis.com/v1/projects/{PROJECT_ID}/locations/{FHIR_LOCATION}/datasets/{FHIR_DATASET_ID}/fhirStores/{FHIR_STORE_ID}/fhir"

# Function to get the FHIR resource URL
def get_gcp_token():
    """Mendapatkan token autentikasi untuk GCP."""
    credentials, _ = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
    auth_req = google.auth.transport.requests.Request()
    credentials.refresh(auth_req)
    return credentials.token

def dapatkan_waktu_sekarang() -> dict:
    """
    Mengembalikan waktu saat ini dalam format Bahasa Indonesia
    dengan zona waktu yang benar (WIB/UTC+7).
    """
    # Tentukan zona waktu Waktu Indonesia Barat (WIB), yaitu UTC+7
    zona_waktu_wib = datetime.timezone(datetime.timedelta(hours=7))

    # Dapatkan tanggal dan waktu saat ini secara spesifik untuk zona waktu WIB
    sekarang = datetime.datetime.now(zona_waktu_wib)
    
    # Format waktu ke dalam string "Jam:Menit:Detik"
    # %H untuk format 24 jam, %I untuk format 12 jam dengan %p (AM/PM)
    waktu_format = sekarang.strftime("%H:%M:%S")
    
    laporan = f"Sekarang pukul {waktu_format} WIB."
    
    return {"status": "success", "report": laporan}

# Function to make FHIR API requests
def make_fhir_request(method: str, resource_path: str, payload: dict = None):
    """Membuat permintaan GET atau POST ke FHIR Datastore."""
    token = get_gcp_token()
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/fhir+json"}
    url = f"{BASE_FHIR_URL}/{resource_path}"
    try:
        if method.upper() == 'GET':
            response = requests.get(url, headers=headers)
        elif method.upper() == 'POST':
            response = requests.post(url, headers=headers, json=payload)
        else:
            raise ValueError("Metode HTTP tidak didukung.")
        
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error saat mengakses FHIR API: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
        return None

# Function to verify patient identity using MRN or name and birthdate
def verifikasi_pasien(nama_depan: Optional[str] = None, nama_belakang: Optional[str] = None, tanggal_lahir: Optional[str] = None, mrn: Optional[str] = None, email: Optional[str] = None):
    """Memverifikasi identitas pasien menggunakan MRN (prioritas) atau kombinasi nama belakang dan tanggal lahir."""
    
    print("\n--- DEBUG: Memulai verifikasi_pasien ---")
    print(f"Input: nama_depan='{nama_depan}', nama_belakang='{nama_belakang}', tanggal_lahir='{tanggal_lahir}', mrn='{mrn}'")
    
    if email:
        print(f"DEBUG: Melakukan verifikasi menggunakan Email: {email}")
        query_path = f"Patient?email={quote(email)}"
        patient_bundle = make_fhir_request('GET', query_path)
        if not patient_bundle or patient_bundle.get("total", 0) == 0:
            return None, {"status": "error", "error_message": f"Pasien dengan email '{email}' tidak ditemukan."}
        if patient_bundle.get("total", 0) > 1:
            return None, {"status": "error", "error_message": "Ditemukan lebih dari satu pasien dengan email tersebut. Mohon gunakan metode verifikasi lain."}
        return patient_bundle["entry"][0]["resource"], None
    
    if mrn and tanggal_lahir:
        print("DEBUG: Melakukan verifikasi menggunakan MRN dan Tanggal Lahir.")
        query_path = f"Patient?identifier={mrn}&birthdate={tanggal_lahir}"
        print(f"DEBUG: URL Permintaan: {BASE_FHIR_URL}/{query_path}")
        patient_bundle = make_fhir_request('GET', query_path)
        print(f"DEBUG: Respons dari server: {patient_bundle}")
        
        if not patient_bundle or patient_bundle.get("total", 0) == 0:
            return None, {"status": "error", "error_message": f"Pasien dengan MRN '{mrn}' dan tanggal lahir '{tanggal_lahir}' tidak ditemukan."}
        return patient_bundle["entry"][0]["resource"], None

    elif nama_depan and tanggal_lahir:
        print("DEBUG: Melakukan verifikasi menggunakan Nama Depan dan Tanggal Lahir.")
        encoded_given_name = quote(nama_depan)
        query_path = f"Patient?given={encoded_given_name}&birthdate={tanggal_lahir}"
        print(f"DEBUG: URL Permintaan: {BASE_FHIR_URL}/{query_path}")
        patient_bundle = make_fhir_request('GET', query_path)
        print(f"DEBUG: Respons dari server: {patient_bundle}")
        
        if not patient_bundle or patient_bundle.get("total", 0) == 0:
            return None, {"status": "error", "error_message": f"Pasien tidak ditemukan dengan data tersebut. Silakan coba verifikasi menggunakan Nomor Rekam Medis (MRN) dan tanggal lahir Anda."}
        
        if patient_bundle.get("total", 0) > 1:
            return None, {"status": "error", "error_message": "Ditemukan data duplikat. Mohon berikan Nomor Rekam Medis (MRN) dan tanggal lahir Anda untuk melanjutkan."}
        
        return patient_bundle["entry"][0]["resource"], None
    
    else:
        print("DEBUG: Gagal verifikasi karena informasi tidak lengkap.")
        return None, {"status": "error", "error_message": "Informasi tidak lengkap. Mohon berikan data verifikasi yang diperlukan."}

def dapatkan_data_pasien_dari_email(email: str) -> dict:
    """Mendapatkan detail data pasien (nama, tanggal lahir, MRN) berdasarkan alamat email."""
    
    print(f"\n--- DEBUG: Memulai dapatkan_data_pasien_dari_email ---")
    print(f"Input: email='{email}'")

    patient_resource, error = verifikasi_pasien(email=email)
    
    if error:
        print(f"DEBUG: Verifikasi via email gagal: {error}")
        return error

    if patient_resource:
        print("DEBUG: Pasien ditemukan via email.")
        # Ekstrak data yang diperlukan
        nama_depan = patient_resource.get("name", [{}])[0].get("given", ["(tidak ada)"])[0]
        nama_belakang = patient_resource.get("name", [{}])[0].get("family", "(tidak ada)")
        tanggal_lahir = patient_resource.get("birthDate", "(tidak ada)")
        
        mrn_value = "(tidak ada)"
        if "identifier" in patient_resource:
            for identifier in patient_resource["identifier"]:
                if identifier.get("type", {}).get("text") == "MRN":
                    mrn_value = identifier.get("value", "(tidak ada)")
                    break
        
        report = (
            f"Data untuk pasien dengan email {email} berhasil ditemukan:\n"
            f"- Nama Depan: {nama_depan}\n"
            f"- Nama Belakang: {nama_belakang}\n"
            f"- Tanggal Lahir: {tanggal_lahir}\n"
            f"- MRN: {mrn_value}"
        )
        print(f"DEBUG: Laporan akhir: {report}")
        return {
            "status": "success",
            "nama_depan": nama_depan,
            "nama_belakang": nama_belakang,
            "tanggal_lahir": tanggal_lahir,
            "mrn": mrn_value,
            "report": report
        }
    
    return {"status": "error", "error_message": "Gagal memproses permintaan."}

In [7]:
instruction_greeting = ("""
    Gunakan alat `dapatkan_waktu_sekarang` untuk mengetahui waktu saat ini.
    Berdasarkan jam sekarang, tentukan salam yang tepat:

    Aturan salam:
    - 04:00–10:59 → "Selamat Pagi"
    - 11:00–14:59 → "Selamat Siang"
    - 15:00–18:59 → "Selamat Sore"
    - 19:00–03:59 → "Selamat Malam"

    1. Jika {verification_status} bernilai 'terverifikasi', Berikan respon dengan format berikut:
        "Selamat Pagi/Siang/Sore/Malam **nama_pasien**! Selamat datang di Asisten Klinis Virtual. \n\n
        "Berikut layanan yang tersedia:\n"
        "- Mendapatkan saran medis\n"
        "- Pendaftaran pasien baru\n"
        "- Membuat janji temu dengan dokter\n"
        "- Mengecek jadwal janji temu\n"
        "- Mencari informasi umum (lokasi, jam operasional, daftar dokter, jadwal praktik dokter)"
                        
    2. Jika pengguna merupakan pasien baru, berikan respon dengan format berikut:
        "Selamat Pagi/Siang/Sore/Malam **nama_pasien**! Selamat datang di Asisten Klinis Virtual. \n\n
        "Apakah Anda ingin melakukan pendaftaran pasien baru ?"
""")

" 0. **Verifikasi Pasien**: Mulai percakapan dengan mengarahkan pengguna ke agen `patient_verification_workflow` untuk melakukan verifikasi identitas.\n"

' 0. **Verifikasi Pasien**: Mulai percakapan dengan mengarahkan pengguna ke agen `patient_verification_workflow` untuk melakukan verifikasi identitas.\n'

# Agent

## Sub Agents

In [10]:
from google.adk.agents import LlmAgent, SequentialAgent
from google.genai import types

verify_patient_identity_agent = LlmAgent(
    name="VerifyPatientIdentityAgent",
    model=model_name,
    description="Agen yang bertugas menyapa dan menyampaikan pesan sebelum proses verifikasi identitas pasien.",
    instruction=("""
        Tugas Anda adalah sebagai berikut:
        1. Gunakan alat `dapatkan_waktu_sekarang` untuk mengetahui waktu saat ini.
    Berdasarkan jam sekarang, tentukan salam yang tepat:

        Aturan salam:
        - 04:00–10:59 → "Selamat Pagi"
        - 11:00–14:59 → "Selamat Siang"
        - 15:00–18:59 → "Selamat Sore"
        - 19:00–03:59 → "Selamat Malam"
        
        2. Kemudian sampaikan pesan berikut kepada pengguna:
        "Selamat Pagi/Siang/Sore/Malam! Saya akan melakukan verifikasi identitas anda terlebih dahulu."
    """),
    generate_content_config=types.GenerateContentConfig(
        temperature=0.2
    ),
    tools=[dapatkan_waktu_sekarang]
)

verification_status_agent = LlmAgent(
    name="VerificationStatusAgent",
    model=model_lite,
    description="Agen yang bertugas memverifikasi identitas pasien.",
    instruction=("""
        Tugas Anda adalah adalah sebagai berikut:\n
        1. Gunakan {email} untuk mendapatkan data pasien.\n
        2. Panggil alat `dapatkan_data_pasien_dari_email` untuk mendapatkan data pasien.
        3. Berdasarkan hasil dari pengecekan data pasien tersebut:\n
            a. Sampaikan respon dibawah ini jika pasien sudah terdaftar: \n
              - 'Terima kasih,**nama_lengkap_pasien**. **Anda sudah terverifikasi**.'\n
            b. Sampaikan respon dibawah ini jika pasien belum terdaftar: \n
              - '**Anda belum terdaftar**.'\n
    """),
    generate_content_config=types.GenerateContentConfig(
        temperature=0.2
    ),
    output_key="verification_status",
    tools=[dapatkan_data_pasien_dari_email]
)

patient_info_agent  = LlmAgent(
    name="PatientInfoAgent",
    model=model_name,
    description="Agen yang menampilkan informasi identitas pasien (Nama, Tanggal Lahir, MRN).",
    instruction=("""
        Tugas Anda adalah sebagai berikut:\n
        1. Bila {verification_status} bernilai 'terverifikasi': 
            - Berikan respon dengan format berikut:\n
                * Nama Depan:  \n
                * Nama Belakang:  \n
                * Tanggal Lahir: hari bulan tahun\n
                * MRN: \n
            - Jangan tampilkan baris field yang kosong.
        2. Jika {verification_status} bernilai 'belum terdaftar', tanyakan apakah pengguna adalah pasien baru?.
    """),
    output_key="patient_info",
    generate_content_config=types.GenerateContentConfig(
        temperature=0.2
    )
)

greeting_agent = LlmAgent(
    name="GreetingAgent",
    model=model_name,
    instruction=instruction_greeting,
    description="Menyapa pengguna sesuai waktu setempat dan menjelaskan layanan apa saja yang bisa dilakukan.",
    tools=[dapatkan_waktu_sekarang]
)

patient_verification_workflow = SequentialAgent(
    name="PatientVerificationWorkflow",
    sub_agents=[verify_patient_identity_agent, verification_status_agent, patient_info_agent, greeting_agent],
    description="Workflow untuk memverifikasi identitas pasien.",
)

## Root Agent

In [11]:
from google.adk.agents import Agent
from google.genai import types

root_agent = Agent(
    name="root_agent",
    model= model_name,
    description="Agen utama yang berfungsi sebagai router untuk mendelegasikan tugas ke sub-agen yang sesuai.",
    instruction = (
    "Anda adalah Asisten Klinis Virtual. Tugas utama Anda adalah memahami permintaan pengguna dan mendelegasikannya kepada sub-agen yang paling sesuai.\n"
    "1. Verifikasi Pasien menggunakan agen `patient_verification_workflow"
    "\n"
),
    sub_agents=[
        patient_verification_workflow
    ]
)

In [14]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

# --- Session Management ---
session_service = InMemorySessionService()
APP_NAME = "healthcare-patient-id"
USER_ID = "user-123"
SESSION_ID = "session-123"

# --- Initial State (Mock Hospital Database) ---
# We create the session with a pre-filled state that represents our hospital's resources.
initial_state = {"email": "julia@email.com"}

runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service
)
print("Runner and SessionService created.")

# --- Interaction Helper Functions ---
async def call_agent_async(query: str):
    """Sends a query to the agent and prints the final response."""
    print(f"\n>>> User Query: {query}")
    content = types.Content(role='user', parts=[types.Part(text=query)])
    final_response_text = "Agent did not produce a final response."
    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response() and event.content and event.content.parts:
            final_response_text = event.content.parts[0].text
            break
    print(f"<<< Agent Response: {final_response_text}")

async def print_current_state(title: str):
    """Retrieves and prints the current session state."""
    print(f"\n--- {title} ---")
    session = await session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    if session:
        import json
        # Pretty print the JSON state
        print(json.dumps(session.state, indent=2))
    print("---------------------------------")

print("Interaction functions are ready.")

Runner and SessionService created.
Interaction functions are ready.


In [16]:
async def run_command_center_conversation():
    # Create the session with our initial hospital state
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state=initial_state
    )
    print("Session created with initial email.")
    await print_current_state("Initial email State")

    # --- Interaction 1: Patient Verification ---
    # This should be delegated to the 'patient_verification_workflow' agent.
    await call_agent_async("halo.")
    await print_current_state("State After Verification")


In [17]:
try:
    print("\n--- Starting Patien Verification Simulation ---")
    await run_command_center_conversation()
except Exception as e:
    print(f"\nAn error occurred. Please check your authentication and Project ID. Error: {e}")


--- Starting Patien Verification Simulation ---
Session created with initial email.

--- Initial email State ---
{
  "email": "julia@email.com"
}
---------------------------------

>>> User Query: halo.


<<< Agent Response: Halo! Ada yang bisa saya bantu?

--- State After Verification ---
{
  "email": "julia@email.com"
}
---------------------------------
